# Multiple Linear Regression
## Homework 2
## Nicholas Thomson

### Import libraries

In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

### Load the dataset
The dataset was downloaded from https://www.kaggle.com/datasets/rush4ratio/video-game-sales-with-ratings

This dataset contains information about steam games. It contains information such as price, estimated owners, supported languages, average play time, genres, etc. I intend to investigate how the variables included in this dataset relate to how many people download a steam game.

In [2]:
df = pd.read_csv('games.csv')

In [3]:
df.head()

,AppID,Name,Release date,Estimated owners,Peak CCU,Required age,Price,DLC count,About the game,Supported languages,...,Average playtime two weeks,Median playtime forever,Median playtime two weeks,Developers,Publishers,Categories,Genres,Tags,Screenshots,Movies
0,20200,Galactic Bowling,"Oct 21, 2008",0 - 20000,0,0,19.99,0,Galactic Bowling is an exaggerated and stylize...,['English'],...,0,0,0,Perpetual FX Creative,Perpetual FX Creative,"Single-player,Multi-player,Steam Achievements,...","Casual,Indie,Sports","Indie,Casual,Sports,Bowling",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
1,655370,Train Bandit,"Oct 12, 2017",0 - 20000,0,0,0.99,0,THE LAW!! Looks to be a showdown atop a train....,"['English', 'French', 'Italian', 'German', 'Sp...",...,0,0,0,Rusty Moyher,Wild Rooster,"Single-player,Steam Achievements,Full controll...","Action,Indie","Indie,Action,Pixel Graphics,2D,Retro,Arcade,Sc...",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
2,1732930,Jolt Project,"Nov 17, 2021",0 - 20000,0,0,4.99,0,Jolt Project: The army now has a new robotics ...,"['English', 'Portuguese - Brazil']",...,0,0,0,Campião Games,Campião Games,Single-player,"Action,Adventure,Indie,Strategy",NaN,https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
3,1355720,Henosis™,"Jul 23, 2020",0 - 20000,0,0,5.99,0,HENOSIS™ is a mysterious 2D Platform Puzzler w...,"['English', 'French', 'Italian', 'German', 'Sp...",...,0,0,0,Odd Critter Games,Odd Critter Games,"Single-player,Full controller support","Adventure,Casual,Indie","2D Platformer,Atmospheric,Surreal,Mystery,Puzz...",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
4,1139950,Two Weeks in Painland,"Feb 3, 2020",0 - 20000,0,0,0.00,0,ABOUT THE GAME Play as a hacker who has arrang...,"['English', 'Spanish - Spain']",...,0,0,0,Unusual Games,Unusual Games,"Single-player,Steam Achievements","Adventure,Indie","Indie,Adventure,Nudity,Violent,Sexual Content,...",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 85103 entries, 0 to 85102
Data columns (total 39 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   AppID                       85103 non-null  int64  
 1   Name                        85097 non-null  object 
 2   Release date                85103 non-null  object 
 3   Estimated owners            85103 non-null  object 
 4   Peak CCU                    85103 non-null  int64  
 5   Required age                85103 non-null  int64  
 6   Price                       85103 non-null  float64
 7   DLC count                   85103 non-null  int64  
 8   About the game              81536 non-null  object 
 9   Supported languages         85103 non-null  object 
 10  Full audio languages        85103 non-null  object 
 11  Reviews                     9743 non-null   object 
 12  Header image                85103 non-null  object 
 13  Website                     394

# Data Preprocessing

I will remove variables that are not useful in our model and variables that contain mostly empty columns.

In [5]:
df_filter = df.drop(['AppID', 'Name', 'Release date', 'About the game', 'Reviews','Header image', 'Website', 'Support url', 'Support email', 'Metacritic url', 'Notes', 'Developers', 'Publishers', 'Screenshots', 'Movies'], axis=1)
#Remove nan values
df_filter = df_filter.dropna()
df_filter.info()

<class 'pandas.core.frame.DataFrame'>
Index: 44 entries, 204 to 77076
Data columns (total 24 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Estimated owners            44 non-null     object 
 1   Peak CCU                    44 non-null     int64  
 2   Required age                44 non-null     int64  
 3   Price                       44 non-null     float64
 4   DLC count                   44 non-null     int64  
 5   Supported languages         44 non-null     object 
 6   Full audio languages        44 non-null     object 
 7   Windows                     44 non-null     bool   
 8   Mac                         44 non-null     bool   
 9   Linux                       44 non-null     bool   
 10  Metacritic score            44 non-null     int64  
 11  User score                  44 non-null     int64  
 12  Positive                    44 non-null     int64  
 13  Negative                    44 non-nu

Here I will map the estimated player count into ascending corresponding integer values. Steam does not give the exact amount of downloads, so an estimation of players is sufficient. This will serve as the dependent variable in our model

In [6]:
df_filter = df_filter.replace(['0 - 0', '0 - 20000', '20000 - 50000', '50000 - 100000', '100000 - 200000', '200000 - 500000', '500000 - 1000000', '1000000 - 2000000', '2000000 - 5000000', '5000000 - 10000000', '10000000 - 20000000', '20000000 - 50000000', '50000000 - 100000000', '100000000 - 200000000'], [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13])

I will replace supported audio and languages into a count variable that tells us how many languages are supported with the game. This is to simplify our model.

In [7]:
lang_count = []
audio_count = []

categories_count = []
genres_count = []
tag_count = []

for x in df_filter['Supported languages']:
    phrase_to_list = x.split(',')
    lang_count.append(len(phrase_to_list))

for x in df_filter['Full audio languages']:
    phrase_to_list = x.split(',')
    audio_count.append(len(phrase_to_list))

#Convert categories to string to split into categories
df_filter['Categories'] = df_filter['Categories'].astype(str)
for x in df_filter['Categories']:
    phrase_to_list = x.split(',')
    categories_count.append(len(phrase_to_list))

#Convert genres to string to split into categories
df_filter['Genres'] = df_filter['Genres'].astype(str)
for x in df_filter['Genres']:
    phrase_to_list = x.split(',')
    genres_count.append(len(phrase_to_list))

#Convert tags to string to split into categories
df_filter['Tags'] = df_filter['Tags'].astype(str)
for x in df_filter['Tags']:
    phrase_to_list = x.split(',')
    tag_count.append(len(phrase_to_list))


df_filter.insert(2, "Supported Languages Count", lang_count, True)
df_filter.insert(2, "Full Audio Languages Count", audio_count, True)
df_filter.insert(2, "Categories Count", categories_count, True)
df_filter.insert(2, "Genres Count", categories_count, True)
df_filter.insert(2, "Tags Count", categories_count, True)

df_filter = df_filter.drop(['Supported languages', 'Full audio languages', 'Categories', 'Genres', 'Tags'], axis = 1)
df_filter.head()

,Estimated owners,Peak CCU,Tags Count,Genres Count,Categories Count,Full Audio Languages Count,Supported Languages Count,Required age,Price,DLC count,...,User score,Positive,Negative,Score rank,Achievements,Recommendations,Average playtime forever,Average playtime two weeks,Median playtime forever,Median playtime two weeks
204,1,1,1,1,1,1,1,0,0.99,0,...,63,54,29,98.0,0,205,0,0,0,0
923,1,1,9,9,9,1,5,18,14.99,8,...,59,61,38,98.0,36,0,481,0,621,0
1737,4,1,7,7,7,2,11,17,0.00,0,...,77,585,167,99.0,0,0,35,0,59,0
2155,1,2,2,2,2,1,1,18,7.99,1,...,68,16,6,99.0,21,197,131,0,236,0
2649,1,0,3,3,3,1,1,0,0.00,0,...,100,10,0,100.0,0,0,0,0,0,0


In [8]:
df_filter.info()

<class 'pandas.core.frame.DataFrame'>
Index: 44 entries, 204 to 77076
Data columns (total 24 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Estimated owners            44 non-null     int64  
 1   Peak CCU                    44 non-null     int64  
 2   Tags Count                  44 non-null     int64  
 3   Genres Count                44 non-null     int64  
 4   Categories Count            44 non-null     int64  
 5   Full Audio Languages Count  44 non-null     int64  
 6   Supported Languages Count   44 non-null     int64  
 7   Required age                44 non-null     int64  
 8   Price                       44 non-null     float64
 9   DLC count                   44 non-null     int64  
 10  Windows                     44 non-null     bool   
 11  Mac                         44 non-null     bool   
 12  Linux                       44 non-null     bool   
 13  Metacritic score            44 non-nu

# Select dependent variable and independent variables
The dependent variable in this case is global sales. The rest of the variables are independent

In [9]:
Y = df_filter['Estimated owners']
X = df_filter.drop(['Estimated owners'],axis=1) # Select all variables except for MEDV

# Split the dataset into training and testing set
The test size of the testing set is 20%. The rest will be used for training, selected randomly

In [10]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=2)

# Create and fit the multiple linear regression model


In [11]:
# Create a linear regression model
model = LinearRegression()

# Fit the model to the training data
model.fit(X_train, Y_train)

LinearRegression()

# Make predictions

In [12]:
Y_pred = model.predict(X_test)

# Evaluate the Model

In [13]:
mse = mean_squared_error(Y_test, Y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(Y_test, Y_pred)

print(f'Mean Squared Error (MSE): {mse:.2f}')
print(f'Root Mean Squared Error (RMSE): {rmse:.2f}')
print(f'R-squared (R²): {r2:.2f}')

Mean Squared Error (MSE): 0.68
Root Mean Squared Error (RMSE): 0.83
R-squared (R²): 0.55


The R-squared value is above 0.5, so the model is significant at predicting values

In [14]:
coefficients = pd.DataFrame({'Feature': X.columns, 'Coefficient': model.coef_})  
print(coefficients) 

                       Feature   Coefficient
0                     Peak CCU  2.273080e-02
1                   Tags Count  5.841945e-03
2                 Genres Count  5.841945e-03
3             Categories Count  5.841945e-03
4   Full Audio Languages Count -5.196359e-03
5    Supported Languages Count  5.472672e-03
6                 Required age -1.703986e-03
7                        Price -6.448562e-03
8                    DLC count -5.285069e-03
9                      Windows -6.869505e-16
10                         Mac  1.390038e-01
11                       Linux  2.452600e-02
12            Metacritic score -3.833732e-03
13                  User score -2.239830e-02
14                    Positive  1.153460e-03
15                    Negative  4.022106e-03
16                  Score rank  4.779043e-01
17                Achievements -3.950724e-04
18             Recommendations -4.847062e-05
19    Average playtime forever  1.874182e-03
20  Average playtime two weeks -2.547750e-04
21     Med

In [15]:
import statsmodels.api as sm

In [16]:
# Add a constant term to the independent variables matrix for the intercept
X = sm.add_constant(X)

# Fit the multiple linear regression model
model = sm.OLS(Y, X).fit()

ValueError: Pandas data cast to numpy dtype of object. Check input data with np.asarray(data).

In [ ]:
summary = model.summary()
print(summary)